In [ ]:
import pandas as pd
import os
import json

In [ ]:
folder_path = "/kaggle/input/uadamage-demining-competition/train/annotations"

In [ ]:
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Read and combine them into one DataFrame
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

In [ ]:
image_folder = "/kaggle/input/uadamage-demining-competition/train/images"

In [ ]:
grouped = combined_df.groupby('image_id').agg(list).to_dict(orient='index')

In [ ]:
json_anot = {"images": [], "annotations": []}

In [ ]:
json_anot["categories"] = [
        {
            "id": 0,
            "name": "Explosives (All sorts of UXO, grenades, etc. that does not fall into AP/AV mine classes)",
            "supercategory": "none"
        },
        {
            "id": 1,
            "name": "Anti-personnel mine",
            "supercategory": "none"
        },
        {
            "id": 2,
            "name": "Anti-vehicle mine",
            "supercategory": "none"
        }
    ]

In [ ]:
def image_json(id, image_id, height, width):
    return  {
            "id": id,
            "file_name": f"{image_id}.jpg",
            "height": height,
            "width": width,
        }

In [ ]:
def annot_json(id, image_id, category_id, bbox, area):
    return {
            "id": id,
            "image_id":image_id,
            "category_id": category_id,
            "bbox": bbox,
            "area": area,
            "segmentation": [],
            "iscrowd": 0
        }

In [ ]:
i = 0
k = 0
for key, value in grouped.items():
    json_anot["images"].append(image_json(
        i, key, value["height"][0], value["width"][0]
    ))
    for j in range(0, len(value["label"])):
        h = value["height"][j]
        w = value["width"][j]
        json_anot["annotations"].append(annot_json(
            k, i, value["label"][j], [value["x"][j], value["y"][j], w, h], w * h
        ))
        k += 1
    i += 1

In [ ]:
with open("annotations.json", "w") as f:
    json.dump(json_anot, f, indent=4)